In [2]:
#This is a starting point
#uncomment below to install merlin library
#!pip install merlin, nvtabular, merlin.models
#I had to downgrade keras to 12.2.0 there maybe issues regarding tensorflow in the future
#run lines below if you have also have an error ab not finding a keras package

#pip uninstall keras
#pip install keras==2.12.0


#These are the same imports from
#https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb
import os
import pandas as pd
import numpy as np
import nvtabular as nvt
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import polars as pl

from sklearn.model_selection import train_test_split

from merlin.models.utils.example_utils import workflow_fit_transform
import merlin.io
import tensorflow as tf

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from nvtabular.ops import *
from merlin.core.utils import download_file
from merlin.schema.tags import Tags

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


### To-do

1. Clean data << Prices on user_interactions are not what they should be... probably is more
2. EDA
3. Transform timestamps
4. Find out why certain things are crashing the model
5. Learn about models and optimizers

In [ ]:
pd.set_option('display.max_columns', None)

x.dtypes

In [61]:
def read_target_sample(file_path, sample_frac = 1, rand = 101, order_weight = 1, added_weight = .5):
    df = pd.read_csv(file_path).dropna() 
    #After cleaning should not need dropna()
    df['TARGET'] = 0
    #Different values for different event_names
    df.loc[df['EVENT_NAME'] == 'order', 'TARGET'] = order_weight
    df.loc[df['EVENT_NAME'] == 'product_added', 'TARGET'] = added_weight

    df = df[df['TARGET'] > 0]
    df = df[df['COUNTRY'] == 'United States']
    df = df.sample(40000, random_state = rand)
    return df

# Cleaning

# TRANSFORM USER DF

# TRANSFORM ITEM DF

In [6]:
def count_product_views(df):
    """
    count num of times each user has viewed a unique STYLE
    
    return: a new column with the count of product views appended to each unique USER_ID and STYLE combination
    """
    return df[df['EVENT_NAME'] == 'product_viewed'].groupby(['USER_ID', 'ITEM_ID']).size().reset_index(name='VIEWS')

In [7]:
def bought_ratio(df):
    """
    for each unique item how many people bought divided by all people or len(unique(USER_ID))
    
    return: a new column with float proportion
    """
    hold = df[df['EVENT_NAME'] == 'order'].groupby('ITEM_ID').size().reset_index(name='BUY_RATIO')
    num_users = df['USER_ID'].nunique()
    return hold

In [8]:
def viewed_ratio(df):
    """
    for each unique item how many people viewed divided by all people or len(unique(USER_ID))
    
    return: a new column with float proportion
    """
    return df[df['EVENT_NAME'] == 'product_viewed'].groupby('ITEM_ID')['USER_ID'].nunique().reset_index(name='viewers')

In [9]:
def cart_ratio(df):
    """
    for each unique item how many people added cart divided by all people or len(unique(USER_ID))
    
    return: a new column with float proportion
    """
    cart_counts = df[df['EVENT_NAME'] == 'add_to_cart'].groupby('ITEM_ID')['USER_ID'].nunique()
    total_users = df['USER_ID'].nunique()
    cart_ratio = cart_counts / total_users
    return cart_ratio

In [10]:
def viewed_to_bought_ratio(df):
    """
    the number of views divided by bought... aka do people need to view it many times before buying it or not
    
    """
    bought_ratio_df = bought_ratio(df)
    viewed_ratio_df = viewed_ratio(df)
    viewed_to_bought_ratio = pd.merge(bought_ratio_df, viewed_ratio_df, on='ITEM_ID', how='left')
    viewed_to_bought_ratio['buy_to_view_ratio'] = viewed_to_bought_ratio['viewers'] / viewed_to_bought_ratio['buyers']
    return viewed_to_bought_ratio.dropna() # only function with nan values i had to drop

In [152]:
from datetime import datetime, timezone
def add_interact_decay(df, halflife = 90):
    now = datetime.now(timezone.utc)
    df.groupby(['USER_ID', 'ITEM_ID']).size().reset_index(name='ViewCount')
    df.groupby(['USER_ID'])['EVENT_TIMESTAMP'].max().reset_index(name='LatestTimestamp')
    df['DaysSince'] = (now - pd.to_datetime(df['EVENT_TIMESTAMP'])).dt.days
    df['DaysSinceDecay'] = np.exp((-1/halflife) * df['DaysSince'])
    df['InteractionDecay'] = df['TARGET'] * df['DaysSinceDecay']
    return df

In [155]:
data = read_target_sample(file_path)
add_interact_decay(data)

,USER_ID,COUNTRY,DERIVED_GENDER_BY_NAME,CLICKSTREAM_EVENTS_TOTAL,FIRST_PURCHASE_AT,FIRST_VISIT_AT,LATEST_VISIT_AT,LATEST_PURCHASE_AT,DAYS_SINCE_FIRST_VISIT,DAYS_SINCE_FIRST_PURCHASE,DAYS_SINCE_LAST_VISIT,DAYS_SINCE_LAST_PURCHASE,RECENT_VISIT,UNIQUE_PURCHASE_COUNT,MULTIPLE_BUYER,EVENT_NAME,EVENT_TIMESTAMP,ITEM_ID,PRICE_INFORMATION,DAYS_SINCE_INTERACTION,AVG_REVIEW_SCORE,STYLE,TAXONOMY_STYLE,COLOR_NAME,PRODUCT_CLASS,PRODUCT_SUBCLASS,TEAM,FRANCHISE,PRODUCT_GROUP,TARGET,DaysSince,DaysSinceDecay,InteractionDecay
271908,f7189a48808567388773c6180a0af784e78bb510dc84ce...,United States,F,980,2020-04-04T19:06:59Z,2021-11-30T00:18:06.124Z,2023-09-03T17:05:18.987Z,2023-09-03T17:05:13Z,680,1285.0,38,38.0,1,1,0,product_added,2023-09-03T17:03:11.022Z,color-womens-tree-breezers-mist,70.0,0.655591,4.596639,womens-tree-breezers,Tree Breezer,Mist (Dark Grey),Footwear,Shoe,Womens,Breezer,Lifestyle,0.5,58,0.524954,0.262477
741013,3ec87884b67184332e348027b13bd096bed46ea14608d2...,United States,M,783,2021-08-26T22:36:00Z,2021-09-12T01:18:04.054Z,2023-08-30T14:29:09.108Z,2023-07-03T00:30:18Z,759,776.0,42,100.0,1,1,0,product_added,2023-07-02T23:49:50.322Z,color-trino-tm-hiders-wheat,4.0,0.325559,3.500000,trino-hiders,Hiders,Wheat,Apparel,Sock,Unisex,Sock,Essentials,0.5,121,0.260684,0.130342
514015,fced1740e599765bbae6e689b5eefea12e863ce4bfb056...,United States,M,15,2020-07-18T19:03:35Z,2023-03-13T01:00:43.736Z,2023-03-13T01:10:45.149Z,2023-03-13T01:10:38Z,212,1180.0,212,212.0,0,1,0,order,2023-03-13T01:10:38Z,color-mens-tree-dashers-blizzard,89.0,0.094843,4.511628,mens-tree-dashers-1,Tree Dasher,Blizzard (White),Footwear,Shoe,Mens,Dasher,Active,1.0,233,0.075103,0.075103
1234995,d6b9779336e96ff6c809f800482214873289a922b0f4db...,United States,F,296,2018-02-09T04:01:34Z,2021-11-16T20:34:43.253Z,2023-02-20T18:28:25.28Z,2023-02-20T18:30:30Z,694,2070.0,233,233.0,0,1,0,order,2023-02-20T18:30:30Z,color-womens-wool-pipers-true-black,110.0,0.075105,4.776119,womens-wool-pipers,Wool Piper,True Black (White),Footwear,Shoe,Womens,Piper,Lifestyle,1.0,253,0.060138,0.060138
1218554,d68f6dae76b2d96bc5631aa2a95ca62ae9aada5a18270d...,United States,F,109,2023-05-09T15:31:36Z,2023-05-09T15:09:24.778Z,2023-07-06T22:03:10.507Z,2023-07-06T22:03:05Z,155,155.0,97,97.0,0,1,0,product_added,2023-05-09T15:29:10.956Z,color-anytime-no-show-sock-blizzard,14.0,0.178672,3.421053,anytime-no-show-sock,Anytime No Show Sock,Blizzard,Apparel,Sock,Unisex,Sock,Essentials,0.5,175,0.143067,0.071533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043918,9630490efc4917ed41f20d7873c5eafea7ab973ca8a79f...,United States,F,720,2022-03-15T16:01:23Z,2022-03-15T15:48:06.986Z,2023-08-30T20:17:07.085Z,2023-08-05T18:08:06Z,575,575.0,42,67.0,1,1,0,product_added,2023-08-04T17:09:36.256Z,color-womens-wool-loungers-natural-white,32.0,0.469753,4.619048,womens-wool-loungers,Wool Lounger,Natural White,Footwear,Slip-On,Womens,Lounger,Lifestyle,0.5,88,0.376146,0.188073
379224,9215c5d78e0063c59903748a018f2ad25332d7c1dd961b...,United States,M,95,2023-07-23T21:49:41Z,2023-07-24T15:50:58.578Z,2023-07-24T16:06:30.249Z,2023-07-24T16:06:27Z,79,80.0,79,79.0,1,1,0,product_added,2023-07-24T16:03:51.488Z,color-anytime-ankle-sock-calm-teal,16.0,0.415709,3.583333,anytime-ankle-sock,Anytime Ankle Sock,Calm Teal,Apparel,Sock,Unisex,Sock,Essentials,0.5,99,0.332871,0.166436
1450843,611ac4a8ef93d1a7b3ece160d96376b1e6f9248ed2336e...,United States,M,91,2019-09-28T20:13:03Z,2022-06-02T17:04:08.28Z,2023-06-26T20:50:16.756Z,2019-09-28T20:13:03Z,496,1474.0,107,1474.0,0,1,0,product_added,2023-03-17T21:59:20.979Z,color-mens-runner-insoles,15.0,0.099153,4.666667,mens-runner-insoles,Runner Insole,Natural Grey,Accessories,Footwear Accessory,Mens,Footwear Accessory,Essentials,0.5,228,0.079394,0.039697
1505913,82f2f028a6fe0e2262eaf3a5ef1a4e4fdd8d76f19a304c...,United States,M,296,2019-06-13T02:47:50Z,2023-06-10T23:42:09.879Z,2023-08-20T23:41:59.123Z,2023-06-11T

## I am going to try and feature engineer on item_df and user_df then merge with negative sampled df.

## Also going to add in testing

In [44]:
def negative_sample(df, multiplier = 1):
    negative_df = df[['USER_ID', 'ITEM_ID', 'TARGET']]
    sections = df.shape[0]//10000
    negative_data = []
    item_map = df['ITEM_ID'].unique()
    rng = np.random.default_rng()
    zero_multiplier = multiplier #https://datascience.stackexchange.com/questions/6939/ratio-of-positive-to-negative-sample-in-data-set-for-best-classification
    #The above datascience stackexchange says 1:1 is a good ratio, i didn't look into it that much tho

    for chunk in np.array_split(df, sections):
        user_id_counts = pd.value_counts(chunk.USER_ID)
        item_ids = []
        for user_id, count in user_id_counts.items():
            item_ids.append(np.random.randint(low = 0, high = item_map.shape[0], size = count*zero_multiplier))
        item_ids = np.concatenate(item_ids)
        negative_data.append(pd.DataFrame({'USER_ID': np.repeat(user_id_counts.index, repeats = user_id_counts.values*zero_multiplier),
                                           'ITEM_ID': item_map[item_ids],
                                           'TARGET': 0}))
    negative_data.append(negative_df)
    sample_data = pd.concat(negative_data)
    sample_data = sample_data.reset_index(drop = True)
    return sample_data

### Merging TEST IN FUTURE

In [96]:
def merge(negative_df, df):
    item_df = df[['STYLE', 'USER_ID', 'PRICE_INFORMATION', 'AVG_REVIEW_SCORE',
              'ITEM_ID', 'TAXONOMY_STYLE', 'COLOR_NAME', 'PRODUCT_CLASS',
              'PRODUCT_SUBCLASS', 'TEAM', 'FRANCHISE', 'PRODUCT_GROUP', 'EVENT_NAME']]
    
    user_df = df[['USER_ID', 'ITEM_ID', 'COUNTRY', 'DERIVED_GENDER_BY_NAME', 'CLICKSTREAM_EVENTS_TOTAL', 'FIRST_PURCHASE_AT', 'FIRST_VISIT_AT',
              'LATEST_VISIT_AT', 'LATEST_PURCHASE_AT']]
    
    df_subset = negative_df.merge(user_df, on = 'USER_ID', how = 'inner').sort_values(['USER_ID', 'TARGET'], ascending=False).drop_duplicates()#.fillna(method='bfill')
    #df_subset = df_subset.merge(item_df, on = ['USER_ID', 'ITEM_ID'], how = 'outer').sort_values(['ITEM_ID', 'TARGET'], ascending=False)#.drop_duplicates(subset = ['USER_ID', 'ITEM_ID']).fillna(method='bfill')
    return df_subset

In [84]:
file_path = '/Users/andrew/Desktop/projects/recsys_data/2023-10-05 9_23pm (2).csv'
df = read_target_sample(file_path, order_weight = 1, added_weight = 0.5)
neg_df = negative_sample(df)

In [108]:
#merged_df = merge(neg_df, pd.read_csv(file_path)) 
df.columns

Index(['USER_ID', 'COUNTRY', 'DERIVED_GENDER_BY_NAME',
       'CLICKSTREAM_EVENTS_TOTAL', 'FIRST_PURCHASE_AT', 'FIRST_VISIT_AT',
       'LATEST_VISIT_AT', 'LATEST_PURCHASE_AT', 'DAYS_SINCE_FIRST_VISIT',
       'DAYS_SINCE_FIRST_PURCHASE', 'DAYS_SINCE_LAST_VISIT',
       'DAYS_SINCE_LAST_PURCHASE', 'RECENT_VISIT', 'UNIQUE_PURCHASE_COUNT',
       'MULTIPLE_BUYER', 'EVENT_NAME', 'EVENT_TIMESTAMP', 'ITEM_ID',
       'PRICE_INFORMATION', 'DAYS_SINCE_INTERACTION', 'AVG_REVIEW_SCORE',
       'STYLE', 'TAXONOMY_STYLE', 'COLOR_NAME', 'PRODUCT_CLASS',
       'PRODUCT_SUBCLASS', 'TEAM', 'FRANCHISE', 'PRODUCT_GROUP', 'TARGET'],
      dtype='object')

In [153]:
pd.set_option('display.max_columns', None)

df.sort_values('DaysSince')

,USER_ID,COUNTRY,DERIVED_GENDER_BY_NAME,CLICKSTREAM_EVENTS_TOTAL,FIRST_PURCHASE_AT,FIRST_VISIT_AT,LATEST_VISIT_AT,LATEST_PURCHASE_AT,DAYS_SINCE_FIRST_VISIT,DAYS_SINCE_FIRST_PURCHASE,DAYS_SINCE_LAST_VISIT,DAYS_SINCE_LAST_PURCHASE,RECENT_VISIT,UNIQUE_PURCHASE_COUNT,MULTIPLE_BUYER,EVENT_NAME,EVENT_TIMESTAMP,ITEM_ID,PRICE_INFORMATION,DAYS_SINCE_INTERACTION,AVG_REVIEW_SCORE,STYLE,TAXONOMY_STYLE,COLOR_NAME,PRODUCT_CLASS,PRODUCT_SUBCLASS,TEAM,FRANCHISE,PRODUCT_GROUP,TARGET,timesince,DaysSince,DaysSinceDecay,InteractionDecay
1506988,89ad0e05180e386fab75cd4b53fd9c3fd07edb37a4f3f2...,United States,F,208,2022-08-28T05:06:40Z,2023-06-25T03:35:42.188Z,2023-08-29T03:14:37.656Z,2023-09-20T04:30:49Z,108,409.0,43,21.0,1,1,0,order,2023-09-20T04:30:49Z,color-womens-wool-runner-mizzles,125.0,0.791891,4.586207,womens-wool-runner-mizzles,Wool Runner Mizzle,Natural Grey,Footwear,Shoe,Womens,Runner,Lifestyle,1.0,41,41,0.634096,0.634096
126028,7dc19042cb0f9cdd964e0bf21e767283ec6331ae943e69...,United States,F,1541,2022-05-15T01:34:45Z,2023-09-10T20:22:59.324Z,2023-09-20T04:55:15.734Z,2023-09-20T04:54:56Z,31,514.0,21,21.0,1,1,0,order,2023-09-20T04:54:56Z,color-womens-tree-dasher-relay-natural-black-b...,99.0,0.791891,4.200000,womens-tree-dasher-relay,Tree Dasher Relay,Natural Black (Buoyant Mint),Footwear,Slip-On,Womens,Dasher,Active,1.0,41,41,0.634096,0.634096
1320477,7b37577124020df6e7050c816de9ecba8e48802877a410...,United States,F,104,2023-09-20T04:52:36Z,2023-09-20T04:33:12.745Z,2023-09-20T06:49:43.213Z,2023-09-20T04:52:36Z,21,21.0,21,21.0,1,1,0,product_added,2023-09-20T04:51:08.836Z,color-anytime-no-show-sock-blizzard,14.0,0.791891,3.421053,anytime-no-show-sock,Anytime No Show Sock,Blizzard,Apparel,Sock,Unisex,Sock,Essentials,0.5,41,41,0.634096,0.317048
374297,785068ea42b3355bac232a3a092a5d5a2b161de1559842...,United States,M,157,2023-06-05T06:23:13Z,2023-09-20T10:25:32.194Z,2023-09-20T10:49:24.202Z,2023-09-20T10:49:16Z,21,128.0,21,21.0,1,1,0,order,2023-09-20T10:49:16Z,color-womens-tree-breezers-jet-black,100.0,0.791891,4.255132,womens-tree-breezers,Tree Breezer,Jet Black (Black),Footwear,Shoe,Womens,Breezer,Lifestyle,1.0,41,41,0.634096,0.634096
932955,c42dc0031848c3eb5bd438c7af4fd2d31bc7c42eda1e2d...,United States,M,106,2020-06-20T13:22:47Z,2023-09-20T09:49:24.963Z,2023-09-20T10:44:50.306Z,2023-09-20T10:44:41Z,21,1208.0,21,21.0,1,1,0,order,2023-09-20T10:44:41Z,color-mens-tree-runners-navy-night,105.0,0.791891,4.692308,mens-tree-runners,Tree Runner,Navy Night (Dark Navy),Footwear,Shoe,Mens,Runner,Lifestyle,1.0,41,41,0.634096,0.634096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940368,114ce80fea2ff197e24c12039aa790a8b42c5505d35b89...,United States,F,20,2023-01-01T13:10:09Z,2023-05-04T14:30:59.996Z,2023-05-04T16:58:40.611Z,2023-05-04T14:34:02Z,160,283.0,160,160.0,0,1,0,order,2023-01-01T13:10:09Z,color-womens-wool-runners-sf-grey,89.0,0.043092,4.500000,womens-wool-runners,Wool Runner,SF Grey,Footwear,Shoe,Womens,Runner,Lifestyle,1.0,303,303,0.034504,0.034504
1131003,ee796467df45973a513470b86eb967b7a8ed3325fdb074...,United States,F,77,2023-01-02T01:25:42Z,2022-12-27T15:59:53.503Z,2023-03-06T23:13:28.157Z,2023-03-06T23:13:21Z,288,282.0,219,219.0,0,1,0,order,2023-01-02T01:25:42Z,color-mens-wool-runners-savanna-dawn,89.0,0.043574,5.000000,mens-wool-runners,Wool Runner,Savanna Dawn,Footwear,Shoe,Mens,Runner,Lifestyle,1.0,303,303,0.034504,0.034504
654565,94249ebf4ca72feae7cb89c02453a09a76072cdfca7d82...,United States,M,186,2022-10-08T20:59:36Z,2022-09-02T14:35:59.674Z,2022-10-08T20:59:42.236Z,2023-01-01T21:59:26Z,404,368.0,368,283.0,0,1,0,order,2023-01-01T21:59:26Z,color-mens-wool-pipers-natural-black,88.0,0.043092,3.937500,mens-wool-pipers,Wool Piper,Natural Black (Black),Footwear,Shoe,Mens,Piper,Lifestyle,1.0,303,303,0.034504,0.034504
992495,ddce04f516368f194c0e10ab3f8461d8dcbca7d22b7239...,United States,F,601,2017-08-15T02:02:23Z,2021-11-16T12:47:37.114Z,2023-0

In [57]:
merged_df.sample(30)

,USER_ID,ITEM_ID,TARGET,COUNTRY,DERIVED_GENDER_BY_NAME,CLICKSTREAM_EVENTS_TOTAL,FIRST_PURCHASE_AT,FIRST_VISIT_AT,LATEST_VISIT_AT,LATEST_PURCHASE_AT,...,PRICE_INFORMATION,AVG_REVIEW_SCORE,TAXONOMY_STYLE,COLOR_NAME,PRODUCT_CLASS,PRODUCT_SUBCLASS,TEAM,FRANCHISE,PRODUCT_GROUP,EVENT_NAME
34218,cc610f345e119a9eeac82bd77f946963c2c920f79a9cb2...,color-womens-wool-runner-up-mizzles-natural-grey,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42562,bfbde239dabe6e77985d770761a2298d384050e9dcb88c...,color-trino-pacers-low-waterfall,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20959,e2b4c3f85658f9a84a586f874872a5d9e0557e4c1404d6...,color-trino-tm-hiders-wheat,0.5,United States,F,545.0,2023-05-23T03:05:33Z,2023-04-15T00:42:09.762Z,2023-09-17T15:35:49.148Z,2023-09-04T09:00:37Z,...,4.0,3.500000,Hiders,Wheat,Apparel,Sock,Unisex,Sock,Essentials,product_added
160414,200d85486b0a6242b484e36325518c9d522bc1af71146b...,color-sugar-sliders-blizzard,0.5,United States,M,9127.0,2020-08-06T09:20:51Z,2021-09-18T04:13:16.35Z,2023-09-01T04:25:34.576Z,2023-08-27T17:00:40Z,...,24.0,4.571429,Sugar Slider,Blizzard,Footwear,Sandal,Unisex,Warm Seasonal,Lifestyle,order
68467,9c6e8d19404e0c8707965d1f688a64935c8ce0adb76408...,color-mens-tree-runners-wheat-dark-beige,0.5,United States,M,1648.0,2019-08-18T13:46:11Z,2021-09-14T11:39:23.696Z,2023-07-30T14:43:04.619Z,2023-07-30T14:42:56Z,...,105.0,4.500000,Tree Runner,Wheat (Dark Beige),Footwear,Shoe,Mens,Runner,Lifestyle,product_added
132885,3de2959c4cb89caf5092795aa1369f13bed67a9830d861...,color-mens-trino-boxer-brief-aspen,0.5,United States,M,180.0,2017-08-23T19:34:19Z,2021-10-28T17:54:13.115Z,2023-06-14T15:38:40.83Z,2023-06-14T15:38:38Z,...,24.0,5.000000,Boxer Brief,Aspen,Apparel,Underwear,Mens,Underwear,Essentials,product_added
125873,497958922795bc6fbe20b31aa4f2388204195121c74d23...,color-anytime-crew-sock-sunny-marigold,1.0,United States,M,8509.0,2019-06-23T21:48:14Z,2021-09-02T01:24:02.089Z,2023-09-19T14:58:16.844Z,2023-08-10T02:35:11Z,...,18.0,3.000000,Anytime Crew Sock,Sunny Marigold,Apparel,Sock,Unisex,Sock,Essentials,product_added
154928,200d85486b0a6242b484e36325518c9d522bc1af71146b...,color-sugar-sliders-blizzard,1.0,United States,M,9127.0,2020-08-06T09:20:51Z,2021-09-18T04:13:16.35Z,2023-09-01T04:25:34.576Z,2023-08-27T17:00:40Z,...,24.0,4.571429,Sugar Slider,Blizzard,Footwear,Sandal,Unisex,Warm Seasonal,Lifestyle,order
176831,0e9dacbfacd3a719f20d0cb33243fdada6f234d5f8df22...,color-mens-wool-runners-natural-black,0.5,United States,F,6.0,2023-01-28T19:35:31Z,2023-01-28T19:15:18.69Z,2023-01-28T19:35:54.625Z,2023-01-28T19:35:31Z,...,110.0,4.587097,Wool Runner,Natural Black (Black),Footwear,Shoe,Mens,Runner,Lifestyle,product_added
182739,04daf183cb3a9982e5e60fcf52fb0c5fa347ba5e68d219...,color-trino-tm-hiders-wheat,0.5,United States,F,163.0,2022-10-22T20:22:25Z,2023-05-06T23:35:56.383Z,2023-05-07T17:43:31.353Z,2023-05-07T00:07:54Z,...,56.0,3.500000,Hiders,Wheat,Apparel,Sock,Unisex,Sock,Essentials,order


In [ ]:
x = pd.read_csv(file_path)
x = x[x['EVENT_NAME'] == 'product_viewed']
x1 = x.groupby(['USER_ID', 'ITEM_ID'])['EVENT_NAME'].count().reset_index(name='ViewCount').sort_values('USER_ID', ascending = False)

In [14]:
#num rows
def train_valid_split_to_parquet(df, ratio = .2):
    data_size = df.shape[0]

    train_split_ratio = int(ratio * data_size)

    train = df[:-train_split_ratio].to_parquet("train.parquet")
    valid = df[-train_split_ratio:].to_parquet("valid.parquet")

# NVTabular

In [15]:
def nvtabular_pipeline():
    categorical_user_features = ['COUNTRY']
    categorical_item_features = ['STYLE', 'TAXONOMY_STYLE', 'COLOR_NAME', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS', 'TEAM', 'FRANCHISE', 'PRODUCT_GROUP']
    user_id = ["USER_ID"] >> Categorify(dtype = "int32", out_path='categories') >> TagAsUserID()
    item_id = ["ITEM_ID"] >> Categorify(dtype = "int32", out_path='categories') >> TagAsItemID()
    item_features = categorical_item_features >> Categorify(dtype = "int32") >> TagAsItemFeatures()
    user_features = categorical_user_features >> Categorify(dtype = "int32") >> TagAsUserFeatures()
    target = ["TARGET"] >> AddTags(["categorical_classification", "target"]) >> Rename(name="INTERACTION_BINARY")
    
    CLICKSTREAM_EVENTS_TOTAL = (
        nvt.ColumnSelector(['CLICKSTREAM_EVENTS_TOTAL'])
        >>FillMissing(1)
        >>LogOp()
        >>Normalize()
        >>LambdaOp(lambda col: col.astype("float32"))
        >>TagAsUserFeatures()
    )

    PRICE_INFORMATION = (
        ['PRICE_INFORMATION']
        >>FillMissing(1)
        >>LogOp()
        >>Normalize()
        >>LambdaOp(lambda col: col.astype("float32"))
        >>TagAsItemFeatures()
    )

    AVG_REVIEW_SCORE = (
        ['AVG_REVIEW_SCORE']
        >>Normalize()
        >>LambdaOp(lambda col: col**2)
        >>TagAsItemFeatures()
    )

    INTERACTION_TIMESTAMP = (
        nvt.ColumnSelector(['DAYS_SINCE_INTERACTION'])
        #>>Normalize()
        >>TagAsUserFeatures()
    )

    DAYS_SINCE_LAST_VISIT = (
        nvt.ColumnSelector(['DAYS_SINCE_LAST_VISIT'])
        >>TagAsUserFeatures()
    )
    categorical_features = (categorical_item_features) #Just a place holder... redo with better categories
    user_cat_features = (user_features)
    label = nvt.ColumnSelector(['TARGET'])
    te_features = categorical_features >> TargetEncoding(label)
    user_feats = user_cat_features >> TargetEncoding(label)
    te_features_norm = te_features >> Normalize() >> LambdaOp(lambda col: col.astype('float32')) >> TagAsItemFeatures()
    user_cat_features_norm = user_feats >> Normalize() >> LambdaOp(lambda col: col.astype('float32')) >> TagAsItemFeatures()
    output = (
        user_id + 
        item_id +
        target +
        CLICKSTREAM_EVENTS_TOTAL + 
        te_features +
        user_feats +
        # INTERACTION_TIMESTAMP +
        # DAYS_SINCE_LAST_VISIT + 
        #PRICE_INFORMATION + #<< THIS CAUSES MODEL TO CRASH AND BURN
        AVG_REVIEW_SCORE
    )

    workflow_fit_transform(output, 'train.parquet', 'valid.parquet', 'integration') 
    train = merlin.io.Dataset(
        os.path.join("integration", "train"), engine="parquet"
    )
    valid = merlin.io.Dataset(
        os.path.join("integration", "valid"), engine="parquet"
    )
    return train,valid

## Model

In [16]:
def preprocessing(order_weight = 1, added_weight = .5):
    df = read_target_sample(file_path, order_weight = 1, added_weight = .5)
    neg_df = negative_sample(df)
    merged_df = merge(neg_df, df) #merge doesn't work right ofc
    train_valid_split_to_parquet(merged_df)
    train, valid = nvtabular_pipeline()
    return train, valid

In [17]:
def ncf_model(train, valid, lr = .0175, epoch = 1):
    model = mm.benchmark.NCFModel(
        train.schema,
        embedding_dim=64,
        mlp_block=mm.MLPBlock([128, 64]),
        prediction_tasks=mm.BinaryOutput(train.schema.select_by_tag(Tags.TARGET).column_names[0]),
    )

    model.compile(optimizer=keras.optimizers.Adam(learning_rate = lr), run_eagerly=False, metrics=[tf.keras.metrics.AUC()]);
    model.fit(train, validation_data=valid, batch_size=1024, epochs = epoch); #Less epochs, more accurate valid... less accurate train
    #print(model.evaluate(valid, batch_size = 1024, return_dict = True))
    return model.predict(valid, batch_size = 1024)

In [18]:
import warnings
warnings.filterwarnings("ignore")

x1, x2 = preprocessing()
pred = ncf_model(x1, x2, epoch = 2)
plt.hist(pred, density = True)
plt.show()

Epoch 1/2
128/128 [==============================] - 3s 12ms/step - loss: 0.3872 - auc: 0.8999 - regularization_loss: 0.0000e+00 - loss_batch: 0.4013 - val_loss: 0.6775 - val_auc: 0.5858 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.7461
Epoch 2/2
128/128 [==============================] - 1s 11ms/step - loss: 0.1702 - auc: 0.9835 - regularization_loss: 0.0000e+00 - loss_batch: 0.1714 - val_loss: 0.8332 - val_auc: 0.5412 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 1.1165


NameError: name 'i' is not defined

# Model

In [ ]:
def dlrm_model():
    model_fb = mm.DLRMModel(
        train.schema,
        embedding_dim = 64,
        top_block = mm.MLPBlock([128, 64, 32]),
        bottom_block = mm.MLPBlock([128, 64]),
    )

    model_fb.compile(optimizer = keras.optimizers.Adam(learning_rate = .0175), metrics=[mm.RecallAt(5), mm.NDCGAt(5)])
    model_fb.fit(train, batch_size = 1024, epochs = 1)
    print(model_fb.evaluate(valid, batch_size = 1024, return_dict = True))

In [ ]:
ncf_model(train, valid)

In [ ]:
preds_2 = model_fb.predict(valid, batch_size = 1024)
plt.hist(preds_2, density = True)